In [1]:
import asyncio
import concurrent.futures
import logging
import sys
import time

import numpy as np

import lsst.sal as sal

class Butler():
    """A totally fake butler"""
    
    def __init__(self):
        pass
    
    def get(self, dataType, dataId):
        if dataType == "raw":
            return np.zeros((100, 200)) + dataId["ccd"]
        else:
            raise RuntimeError("Unknown dataType: %s" % dataType)
            
butler = Butler()

# Configure logging to show the name of the thread where the log message originates.
logging.basicConfig(
    level=logging.INFO,
    format='%(threadName)10s %(name)18s: %(message)s',
    stream=sys.stdout,
)

In [2]:
async def turnOnLaser():
    """Ask SAL to turn on the laser

    Can block;  could rewrite to use asyncio (cf. waitForCompletion_cameraIntegrate)
    """
    myData = sal.camera_command_turnOnLaserC()
    cmdId = cameraMgr.issueCommand_turnOnLaser(myData)

    return await cameraMgr.awaitForCompletion_turnOnLaser(cmdId, timeout=10)    
    
async def turnOffLaser():
    """Turn off the laser"""

    await asyncio.sleep(0.1)

    print("%s Laser is off" % (time.asctime(),), file=sys.stderr)
    sys.stderr.flush()

async def setFilter(filterName):
    """Ask SAL to set the filter"""
    myData = sal.camera_command_setFilterC(filterName)
    cmdId = cameraMgr.issueCommand_setFilter(myData)

    return await cameraMgr.awaitForCompletion_setFilter(cmdId, timeout=10)
    
async def setLaserWavelength(wavelength):
    """Set the laser wavelength"""

    print("%s Setting wavelength to %g" % (time.asctime(), wavelength), file=sys.stderr)
    sys.stderr.flush()
    
    await asyncio.sleep(2.0)

    if wavelength > 1000:
        raise RuntimeError("%g is too large (> 1000nm)" % wavelength)

    print("%s Set laser to %g" % (time.asctime(), wavelength), file=sys.stderr)
    sys.stderr.flush()

async def checkDiodeCurrent(sem, dt=0.5):
    """Check the diode current every dt seconds, until sem is released
    
    Returns the list of measured currents
    """
    diodeCurrents = []
    while True:
        if not sem.locked():
            break
            
        diodeCurrents.append(time.clock()%10)

        await asyncio.sleep(dt)
        
    return diodeCurrents

async def cameraIntegrate(sem, expTime, pollTime=0.1):
    """Integrate for the specified exposure time
    
    sem is a semaphore used to indicate that the exposure is finished    
    """
    myData = sal.camera_command_cameraIntegrateC(expTime)
    cmdId = cameraMgr.issueCommand_cameraIntegrate(myData)
    
    ret = await cameraMgr.waitForCompletion_cameraIntegrate(cmdId)
    sem.release()
    
    return ret

In [3]:
def makeNamedTask(coro, name=None, loop=None):
    """Return a task to run in loop, with specified name
    
    If not specified, name is the name of the input coroutine
    """
    if loop is None:
        loop = asyncio.get_event_loop()
    if name is None:
        name = coro.__name__

    task = loop.create_task(coro)
    task.name = name
    
    return task

In [4]:
print("Initialising SAL\n", file=sys.stderr)
cameraMgr = sal.SAL_camera()

cameraMgr.salCommand("setFilter")    # Enable the camera commands
cameraMgr.salCommand("turnOnLaser")
cameraMgr.salCommand("cameraIntegrate")

cameraMgr.salEvent("exposureId")
#
# Turn on laser and set filter
#
wavelength = 656 if True else 1260   # N.b. > 1000 will raise an exception

print("%s Step 0" % (time.asctime()), file=sys.stderr)

loop = asyncio.get_event_loop()
    
try:
    done, pending = loop.run_until_complete(asyncio.wait(
        [
            setFilter('g'),
            turnOnLaser(),
            setLaserWavelength(wavelength),
        ]))
    for task in done:
        task.result()
    #
    # Take an exposure
    #
    cameraSem = asyncio.Semaphore(0)
    exposureIdEv = sal.camera_logevent_exposureIdC()

    print("\n%s Step 1" % (time.asctime()), file=sys.stderr)
    done, pending = loop.run_until_complete(asyncio.wait(
        [
            makeNamedTask(checkDiodeCurrent(cameraSem, 0.5)),
            cameraIntegrate(cameraSem, 2),
        ]))
    for task in done:
        res = task.result()
        if hasattr(task, "name") and task.name == "checkDiodeCurrent":
            diodeCurrents = res

    print("Diode currents:", diodeCurrents, file=sys.stderr)
    #
    # Retrieve data
    #
    print("\n%s Step 2" % (time.asctime()), file=sys.stderr)
    if cameraMgr.getEvent_SummaryState(exposureIdEv) != 0:
        raise RuntimeError("Visit number is not available")

    dataId = dict(visit=exposureIdEv.visit, ccd=1)
    exp = butler.get("raw", dataId)

    print("Mean of exposure is %.1f" % exp.mean(), file=sys.stderr)
finally:
    #
    # Clean up
    #
    print("\n%s Step 3" % (time.asctime()), file=sys.stderr)
    done, pending = loop.run_until_complete(asyncio.wait(
        [
            turnOffLaser(),
        ]))
    for task in done:
        task.result()

    cameraMgr.salShutdown()
    print("\nSAL is shutdown", file=sys.stderr)

Initialising SAL

Tue Apr 24 17:24:34 2018 Step 0
Tue Apr 24 17:24:34 2018 turning laser on
Tue Apr 24 17:24:34 2018 Setting g
Tue Apr 24 17:24:34 2018 Setting wavelength to 656
Tue Apr 24 17:24:35 2018 Set filter to g
Tue Apr 24 17:24:36 2018 laser is warm
Tue Apr 24 17:24:36 2018 Set laser to 656

Tue Apr 24 17:24:36 2018 Step 1
Tue Apr 24 17:24:36 2018 opening shutter
Tue Apr 24 17:24:38 2018 Shutter closed
Diode currents: [2.844588, 2.851572, 2.853472, 2.855198, 2.85705]

Tue Apr 24 17:24:38 2018 Step 2
Mean of exposure is 1.0

Tue Apr 24 17:24:38 2018 Step 3
Tue Apr 24 17:24:38 2018 Laser is off

SAL is shutdown
